# Summary
This notebook contains the code for generating the data used in the synthetic experiments.

In [1]:
import sys
sys.path.insert(0, r"..\code")

import jax
import jax.numpy as jnp
import numpy as np

from jax import random
from utils import save_obj, load_obj
from jax.numpy.linalg import pinv

In [2]:
data_dir="../data/"
ts_dir="../data/TimeSeriesAAL/" 
processed="../data/processed/"
diffreg="../data/processed/diff_regions/"

In [3]:
replace=False

In [4]:
def backbone(K, density, seed):
    key = random.PRNGKey(seed)
    mask=jax.random.bernoulli(key, density, shape=(1,K,K))
    triu = jnp.triu(jnp.ones((1,K,K)), k=1)
    return mask*triu

def SEM(K, T, density, S=1, with_backbone=True, density_backbone=.3, seed=42, seed_backbone=24):
    """
    This function generates data from a SEM, 
    with strictly upper triangular matrix. 
    """
    
    key = random.PRNGKey(seed)
    weight_matrix=jax.random.uniform(key, shape=(S,K,K), minval=-1, maxval=1)
    new_key, subkey = random.split(key)
    
    mask=jax.random.bernoulli(subkey, density, shape=(S*K*K,)).reshape(S,K,K)
    triu = jnp.triu(jnp.ones((1,K,K)), k=1)
    
    Pi = mask*triu

    if with_backbone:
        backbone_1kk= backbone(K, density_backbone, seed_backbone)
        Pi = jnp.sign(Pi+backbone_1kk)

    causal_matrix=Pi*weight_matrix

    new_key1, subkey1 = random.split(new_key)
    I_aug=jnp.ones((S,1,1))*jnp.eye(K)
    #generate noise Z with dim (S,K,T)
    Z=jax.random.normal(subkey1, shape=(S,K,T))
    #compute M=(1-C.T)^-1
    vpinv=jax.vmap(pinv,0,0)
    M=vpinv(I_aug - jnp.transpose(causal_matrix, (0,2,1)))
    #generate X=MZ
    X=jnp.einsum("skp,spt->skt",M,Z) 

    if backbone:
        return X, causal_matrix, jnp.squeeze(backbone_1kk, axis=0)
    return X, causal_matrix

In [5]:
S,K,T=100,10,1200
density=.5
density_backbone=.25
nsims = 50

seed_backbone=24
seeds=np.arange(50)

if replace:
    data = dict()
    data['info']="S: # of subjects;\nK: # of ts;\nT: length of ts;\nX_skt: data generated from the causal structure;\nC_skk: causal structure;\nB_kk: backbone."

    for sim in range(nsims):
        data[sim]=dict()
        X, causal_matrix, backbone_kk = SEM(K=K, T=T, density=density, S=S, with_backbone=True, density_backbone=density_backbone, seed=seeds[sim], seed_backbone=seed_backbone)
        data[sim]['(seed, seed_backbone)']=(seeds[sim], seed_backbone)
        data[sim]['X_skt'] = np.array(X)
        data[sim]['C_skk'] = np.array(causal_matrix)
        data[sim]['B_kk'] = np.array(backbone_kk)

    save_obj(data, "2.1_data", processed)
else:
    data=load_obj("2.1_data", processed)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
